In [1]:
import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import os
import matplotlib.pyplot as plt
import numpy as np
from subprocess import call
import warnings
import random
import tensorflow as tf

warnings.filterwarnings("ignore")  # specify to ignore warning messages
from keras.optimizers import SGD
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential
from pandas import DataFrame
from keras.layers import TimeDistributed
from keras.layers import Dropout
from keras.utils import plot_model
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.layers import Lambda
from keras import backend as K
from netCDF4 import Dataset
from netCDF4 import getlibversion
import glob
from keras.models import load_model
import keras

#os.environ['CUDA_VISIBLE_DEVICES'] ="0"
#tf.config.set_visible_devices([], 'GPU')

arr_dir = '../../model_collocated_10d_filled/'

# define model
activ = 'tanh'#'softsign'#
opt='Adam'
pat=30
n_epochs = 1000
val_split=.15
dropout_fraction=.2
n_units1 =35
n_units2 =35
# n_units3 =n_unit
batch_size = 16 #16
batch_size_val = batch_size//4
n_depth = 46
n_var_in = 10
n_var_out = 2
learning_rate = 0.001

2024-05-08 14:47:17.021248: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-08 14:47:17.054207: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 14:47:17.054237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 14:47:17.055300: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 14:47:17.061161: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def PermaDropout(rate):
    return Lambda(lambda x: Dropout(x, rate=rate))

model = Sequential([
    Input(shape=[n_depth, n_var_in], dtype = tf.float32),
    LSTM(n_units1, return_sequences=True, activation=activ),
    Dropout(dropout_fraction),
    LSTM(n_units2, return_sequences=True, activation=activ),
    Dropout(dropout_fraction),
    TimeDistributed(Dense(n_var_out))
    #TimeDistributed(Dense((n_depth,n_var_out)))
])

model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=1e-5), metrics=['accuracy'])
dummy_batch_X = np.zeros((batch_size, n_depth, n_var_in))  # Replace with your actual batch data
# Call the model on the dummy batch to build the model
model(dummy_batch_X)

model.summary()

2024-05-08 13:52:01.297798: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2721 MB memory:  -> device: 0, name: NVIDIA T1000, pci bus id: 0000:65:00.0, compute capability: 7.5


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 46, 35)            6440      
                                                                 
 dropout (Dropout)           (None, 46, 35)            0         
                                                                 
 lstm_1 (LSTM)               (None, 46, 35)            9940      
                                                                 
 dropout_1 (Dropout)         (None, 46, 35)            0         
                                                                 
 time_distributed (TimeDist  (None, 46, 2)             72        
 ributed)                                                        
                                                                 
Total params: 16452 (64.27 KB)
Trainable params: 16452 (64.27 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

2024-05-08 13:52:01.782160: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


In [2]:
import pandas as pd
import os 
import json 

def load_data_from_feather(daily_ds):
    if isinstance(daily_ds, bytes):
        daily_ds = daily_ds.decode('utf-8')    
    df = pd.read_feather('../../model_collocated_10d_filled/'+daily_ds)
    asal_height_cols = [col for col in df.columns if col.startswith('ASAL_height')]
    ctemp_height_cols = [col for col in df.columns if col.startswith('CTEMP_height')]
    
    other_vars = ['LATITUDE', 'LONGITUDE', 'SSS', 'SST', 'SSH', 'MLD', 'UO', 'VO']
    
    new_df = df[other_vars]
    
    height_dfs = []
    for asal_col, ctemp_col in zip(asal_height_cols, ctemp_height_cols):
        height_df = new_df.copy()
        height_df['HEIGHT'] = asal_col.split('_height')[-1]
        height_df['ASAL'] = df[asal_col]  
        height_df['CTEMP'] = df[ctemp_col] 
        height_dfs.append(height_df)
    
    final_df = pd.concat(height_dfs, ignore_index=True)
    final_df = final_df.dropna()

    pivot_table = final_df.pivot_table(index=['LATITUDE', 'LONGITUDE', 'HEIGHT'], values=['SSS', 'SST', 'SSH', 'MLD', 'UO', 'VO', 'ASAL', 'CTEMP'])

    filename = os.path.basename(daily_ds)
    date_str = filename.split('.')[0] 
    date = pd.to_datetime(date_str, format='%Y%m%d')
    day_of_year = date.dayofyear

    pivot_table['DATE'] = day_of_year
    pivot_table = pivot_table.reset_index()
    pivot_table = pivot_table.astype(float)
    pivot_table = pivot_table[['LATITUDE', 'LONGITUDE', 'HEIGHT', 'SSS', 'SST', 'SSH', 'MLD', 'UO', 'VO', 'DATE', 'ASAL', 'CTEMP']]

    cols = ['LATITUDE', 'LONGITUDE', 'SSS', 'SST', 'SSH', 'MLD', 'UO', 'VO', 'DATE', 'HEIGHT']

    groups = pivot_table.groupby(['LATITUDE', 'LONGITUDE'])
    
    arrays = []
    
    n_row = 46
    
    with open('min_max_values.json', 'r') as f:
        min_max_values = json.load(f)
        
    for name, group in groups:
        if len(group) == n_row:
            group_sorted = group.sort_values(by='HEIGHT')
            for column in min_max_values['min_values'].keys():
                min_value = min_max_values['min_values'][column]
                max_value = min_max_values['max_values'][column]
                group_sorted[column] = (group_sorted[column] - min_value) / (max_value - min_value)

            array = group_sorted.values
            if (np.any(array)):
                arrays.append(array)
    
    all_arr = np.stack(arrays)
    X = all_arr[:,:,0:10]
    y = all_arr[:,:,10:12]
    return X, y


def data_generator(feather_files, batch_size):
    for file in feather_files:
        X, y = load_data_from_feather(file)
        num_samples = X.shape[0]
        num_batches = num_samples // batch_size
        for i in range(num_batches):
            start_idx = i * batch_size
            end_idx = (i + 1) * batch_size
            batch_X = X[start_idx:end_idx]
            batch_y = y[start_idx:end_idx]            
            yield batch_X, batch_y

In [3]:
import os 

feather_files = [f for f in os.listdir(arr_dir) if f.endswith('.feather') and os.path.isfile(os.path.join(arr_dir, f))]

start_date = '20100101'
end_date = '20221231'

random.seed(56)
random.shuffle(feather_files)

# Set 0.8 training and 0.2 testing
split_point = int(len(feather_files) * 0.8)  

training_dates = feather_files[:split_point]
testing_dates = feather_files[split_point:]

split_point_val = int(len(training_dates)*0.9)
validation_dates = training_dates[split_point_val:]
training_dates = training_dates[:split_point_val]

dataset_training = tf.data.Dataset.from_generator(
    data_generator,
    args=[training_dates, batch_size],  
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, n_depth, n_var_in], [None, n_depth, n_var_out])
)

dataset_training = dataset_training.repeat()

# Convert string dates to datetime objects
dataset_validation = tf.data.Dataset.from_generator(
    data_generator,
    args=[validation_dates, batch_size_val],  # Pass batch_size as an argument
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, n_depth, n_var_in], [None, n_depth, n_var_out])
)

dataset_validation = dataset_validation.repeat()

for batch_X, batch_y in dataset_training.take(1):
    print("Input shape:", batch_X.shape)
    print("Output shape:", batch_y.shape)
    
for batch_X, batch_y in dataset_validation.take(1):
    print("Input shape:", batch_X.shape)
    print("Output shape:", batch_y.shape)


Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead


2024-05-08 14:47:22.272206: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2720 MB memory:  -> device: 0, name: NVIDIA T1000, pci bus id: 0000:65:00.0, compute capability: 7.5


Input shape: (16, 46, 10)
Output shape: (16, 46, 2)
Input shape: (4, 46, 10)
Output shape: (4, 46, 2)


In [ ]:
import math
from keras.callbacks import ReduceLROnPlateau

# fit model
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=pat)
steps_per_epoch = 8000

validation_steps = 2000

history = model.fit(
    dataset_training,
    epochs=n_epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=dataset_validation,
    validation_steps=validation_steps,
    verbose=1, callbacks=[es])
model.save('LSTM_512_512.h5')
print("Saved model to disk")

train = history.history['loss']
val = history.history['val_loss']

# plot train and validation loss across multiple runs
plt.plot(train, color='blue', label='train')
plt.plot(val, color='orange', label='validation')
plt.title('LSTM ' + str(n_units1) + '-' + str(n_units2) + ' model')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show(block=False)
plt.savefig('LSTM_' + str(n_units1) + '_' + str(n_units2)+ '_loss.eps', dpi=150)
plt.close()